Tìm hiểu thư viện apyori (https://pypi.org/project/apyori) để thực hiện tìm tập phổ biến từ dữ liệu 
mua hàng tại siêu thị. Tham khảo bài hướng dẫn https://www.kaggle.com/code/rockystats/apriori-algorithm-or-market-basket-analysis 

1. import libraries

In [2]:
import pandas as pd
from apyori import apriori

2. ĐỌC & LÀM SẠCH DỮ LIỆU data-2.csv (Ecommerce Dataset)

In [4]:
ecommerce = "Dataset/data-2.csv"

# một số file dùng latin-1 hoặc ISO-8859-1
df = pd.read_csv(ecommerce, encoding="latin-1")

print("Các cột có trong data-2.csv:")
print(df.columns)


Các cột có trong data-2.csv:
Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')


In [8]:
# Giữ lại các cột cần thiết 
cols_map = {
    "InvoiceNo": "InvoiceNo",
    "Description": "Description",
    "Quantity": "Quantity"
}

# đảm bảo các cột tồn tại
for c in cols_map.values():
    if c not in df.columns:
        raise ValueError(f"Thiếu cột '{c}' trong data-2.csv")

df = df[list(cols_map.values())].copy()


In [10]:
# --- làm sạch dữ liệu---
# bỏ dòng thiếu InvoiceNo hoặc Description
df = df.dropna(subset=["InvoiceNo", "Description"])

# đưa InvoiceNo về string
df["InvoiceNo"] = df["InvoiceNo"].astype(str)

# bỏ các hóa đơn bị hủy nếu có dạng 'Cxxxxx'
df = df[~df["InvoiceNo"].str.startswith("C")]

# giữ Quantity > 0
df = df[df["Quantity"] > 0]

# chuẩn hóa tên sản phẩm
df["Description"] = df["Description"].str.strip()
df = df[df["Description"] != ""]

print("Số dòng sau khi làm sạch:", len(df))


Số dòng sau khi làm sạch: 530693


3. CHUYỂN SANG DANH SÁCH GIAO DỊCH CHO apyori

In [11]:
transactions = (
    df.groupby("InvoiceNo")["Description"]
      .apply(list)
      .tolist()
)

print("Số giao dịch:", len(transactions))
print("Ví dụ 1 giao dịch:", transactions[0][:10])

Số giao dịch: 20136
Ví dụ 1 giao dịch: ['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER', 'KNITTED UNION FLAG HOT WATER BOTTLE', 'RED WOOLLY HOTTIE WHITE HEART.', 'SET 7 BABUSHKA NESTING BOXES', 'GLASS STAR FROSTED T-LIGHT HOLDER']


 4. CHẠY APRIORI VỚI apyori TRÊN data-2.csv

In [ ]:
min_support = 0.005      # 0.5% giao dịch
min_confidence = 0.2
min_lift = 1.0
min_length = 2           # ít nhất 2 sản phẩm
max_length = 3           # tối đa 3 sản phẩm 

results = list(
    apriori(
        transactions,
        min_support=min_support,
        min_confidence=min_confidence,
        min_lift=min_lift,
        min_length=min_length,
        max_length=max_length
    )
)

print("Số tập phổ biến / luật tìm được:", len(results))

Số tập phổ biến / luật tìm được: 10397


5. TÁCH TẬP PHỔ BIẾN & LUẬT KẾT HỢP -> DATAFRAME

In [13]:
frequent_items = []
rules_list = []

for item in results:
    itemset = list(item.items)
    support = item.support

    frequent_items.append({
        "itemset": ", ".join(itemset),
        "support": support
    })

    for stat in item.ordered_statistics:
        base = list(stat.items_base)
        add = list(stat.items_add)

        if len(base) == 0 or len(add) == 0:
            continue

        rules_list.append({
            "antecedent": ", ".join(base),
            "consequent": ", ".join(add),
            "support": support,
            "confidence": stat.confidence,
            "lift": stat.lift
        })

frequent_df = pd.DataFrame(frequent_items).drop_duplicates()
rules_df = pd.DataFrame(rules_list).drop_duplicates()

# sắp xếp cho dễ xem
frequent_df = frequent_df.sort_values(by="support", ascending=False)
rules_df = rules_df.sort_values(by=["lift", "confidence"], ascending=False)

print("\n TOP 10 TẬP PHỔ BIẾN ")
print(frequent_df.head(10))

print("\nTOP 10 LUẬT KẾT HỢP ")
print(rules_df.head(10))


 TOP 10 TẬP PHỔ BIẾN 
                                                itemset   support
2615   JUMBO BAG RED RETROSPOT, JUMBO BAG PINK POLKADOT  0.040971
1868  GREEN REGENCY TEACUP AND SAUCER, ROSES REGENCY...  0.038141
2710    JUMBO BAG RED RETROSPOT, JUMBO STORAGE BAG SUKI  0.035956
2708  JUMBO BAG RED RETROSPOT, JUMBO SHOPPER VINTAGE...  0.033770
3259   LUNCH BAG  BLACK SKULL., LUNCH BAG RED RETROSPOT  0.031834
257   ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKELI...  0.031784
1853  GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY ...  0.031436
3395   LUNCH BAG RED RETROSPOT, LUNCH BAG PINK POLKADOT  0.030095
4139  PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY ...  0.029748
2355  JUMBO  BAG BAROQUE BLACK WHITE, JUMBO BAG RED ...  0.029052

TOP 10 LUẬT KẾT HỢP 
                                              antecedent  \
11337  DOLLY GIRL CHILDRENS CUP, SPACEBOY CHILDRENS BOWL   
11332                          DOLLY GIRL CHILDRENS BOWL   
1572                 CHRISTMAS TREE DECORATION WI

6. LƯU KẾT QUẢ 

In [15]:
import os

# Tạo thư mục result
os.makedirs("result", exist_ok=True)

# Lưu vào thư mục result
frequent_df.to_csv("result/frequent_itemsets_data2_apyori.csv", index=False)
rules_df.to_csv("result/association_rules_data2_apyori.csv", index=False)

print("\nĐã lưu file trong thư mục result/:")
print(" - result/frequent_itemsets_data2_apyori.csv")
print(" - result/association_rules_data2_apyori.csv")


Đã lưu file trong thư mục result/:
 - result/frequent_itemsets_data2_apyori.csv
 - result/association_rules_data2_apyori.csv
